In [0]:
catalog_name = "dataops_prd"
schema_name = "libs"

In [0]:
%sql
CREATE FUNCTION dataops.libs.create_catalog(catalog_name STRING)
  RETURNS STRING
  LANGUAGE PYTHON
  AS $$
    catalog_name = catalog_name.replace("-", "_")
    if catalog_name not ends with "_prd", ""
    if catalog_name not in spark.catalog.listDatabases():
      spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog_name}")
    def create_catalog(catalog_name):
      return "Hello " + name + "!"

    return greet(s) if s else None
  $$

In [0]:
def create_catalog(catalog_name,spark=spark):
  catalog_name = catalog_name.replace("-", "_")
  spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog_name}")
  spark.sql(f)
  spark.sql(f"USE CATALOG {catalog_name}")

In [0]:
def create_curr_wksp_tb(catalog_name:str, schema_name:str):
    from dbruntime.databricks_repl_context import get_context
    from IPython.display import display, HTML

    id = get_context().workspaceId
    workspace_url = spark.conf.get('spark.databricks.workspaceUrl')
    catalog_name = catalog_name.replace("-", "_")
    schema_name = schema_name.replace("-", "_")
    if not catalog_name.endswith("_prd"):
        catalog_name = catalog_name + "_prd"
    if catalog_name not in spark.sql("SHOW CATALOGS").selectExpr("collect_list(catalog)").first()[0]:
        spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog_name}")
    spark.sql(f"USE CATALOG {catalog_name}")
    if schema_name not in spark.sql("SHOW SCHEMAS").selectExpr("collect_list(databaseName)").first()[0]:
        spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog_name}.{schema_name}")
    spark.sql(f"CREATE OR REPLACE TABLE {catalog_name}.{schema_name}.current_workspace AS SELECT {id} AS workspace_id, '{workspace_url}' AS workspace_url")
    html_link = f'<a href="https://{workspace_url}/explore/data/{catalog_name}/{schema_name}/current_workspace" target="_blank">Table {catalog_name}.{schema_name}.current_workspace created</a>'
    display(HTML(html_link))
create_curr_wksp_tb("dataops","libs")

In [0]:
%sql
CREATE OR REPLACE FUNCTION dataops_prd.libs.get_workspace_id()
RETURNS STRING
COMMENT 'Returns current workspace id'
LANGUAGE SQL
RETURN 
SELECT workspace_id FROM dataops_prd.libs.current_workspace LIMIT 1
;

In [0]:
%sql
SELECT wl.* FROM dataops_prd.libs.current_workspace cw
inner join system.access.workspaces_latest wl on cw.workspace_id = wl.workspace_id